In [3]:
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
conn = psycopg2.connect(dbname='sociolinguistic', user='app', 
                        password='rfrfyl.babaloos', host='gisly.net')

Интересующие нас таблицы:

1. sociolinguistic_person
    * id
    * year_born
    * place_born_id
2. sociolinguistic_personplaces
    * person_id
    * place_id
    * is_present
    * year_from
    * year_to
3. sociolinguistic_personrelation
     * person1_id
     * person2_id
     * relation_id
4. sociolinguistic_relation
    * id
    * name
5. sociolinguistic_questionnaire
    * place_id
    * person_id
    * year
6. sociolinguistic_place
    * id
    * latitude
    * longitude

### Проблема: недостаточно данных по таблице sociolinguistic_personplaces

В базе присутствуют данные о выездах отдельных людей. Однако использовать их трудно.

* нужно вручную валидировать данные и перенести некоторые из description: сделано
* многие поля с заполненными годами относятся к армии, тюрьме, пансионатам. Имеет ли смысл учитывать такие данные?
* непонятно, откуда переезд. Можно попробовать прикинуть по годам, но данных мало.
* 800+ строк, заполнено годами только 84

Однако в таблице есть данные о нахождении респондента в конкретном месте на момент опроса. Эту информацию мы могли бы использовать.

### Проблема: не проставлен год рождения

У многих людей, особенно старшего поколения, не проставлена дата рождения. Эта информация понадобится нам для того, чтобы определить, в какой примерно период был совершен переезд. Опираясь на даты рождения младших родственников, мы можем прикинуть примерный год рождения, взяв за одно поколение 30 лет. Для удобства создадим view с тремя столбцами: id, year_born, is_presice.

6. analytic_person_places
    * id,
    * year_born
    * is_precise

            select distinct q2.id,
            case
            when status = 'child' then q2.child_year_born
            when status = 'parent'
            and q2.parent_year_born_precise is not null then q2.parent_year_born_precise
            when status = 'parent'
            and q2.parent_year_born_precise is null then q2.parent_year_born_appr
            when status = 'grandparent'
            and q2.grandparent_year_born_precise is not null then q2.grandparent_year_born_precise
            when status = 'grandparent'
            and q2.grandparent_year_born_precise is null then q2.grandparent_year_born_appr

            end as year_born,

            case
            when status = 'child' then true
            when status = 'parent' and q2.parent_year_born_precise is not null then  true 
            when status = 'grandparent' and q2.grandparent_year_born_precise is not null then  true 
            else false end as is_precise

            from 
            (
            select sp.id,
            case when sp.id = q.child_id then 'child'
            when sp.id = q.parent_id then 'parent'
            else 'grandparent' end as status,
            q.child_year_born, q.parent_year_born_precise, q.parent_year_born_appr,
            q.grandparent_year_born_precise, q.grandparent_year_born_appr
            from sociolinguistic_person sp,
            (
            SELECT 
                  a.id as child_id, 
                  b.id as parent_id, 
                  c.id as grandparent_id,
                  a.year_born AS child_year_born,
                  b.year_born AS parent_year_born_precise, 
                  (a.year_born - 30) AS parent_year_born_appr,
                   c.year_born AS grandparent_year_born_precise, 
                  (a.year_born - 60) AS grandparent_year_born_appr,
                  r.name as r1_name, 
                  r2.name as r2_name
              from sociolinguistic_person as a
              join sociolinguistic_personrelation as pr
              on a.id = pr.person1_id 
              join sociolinguistic_person as b
              on b.id = pr.person2_id 
              join sociolinguistic_relation as r
              on r.id = pr.relation_id 

              left join sociolinguistic_personrelation as pr2
              on pr2.person1_id = b.id
              left join sociolinguistic_person as c
              on pr2.person2_id  = c.id 
              join sociolinguistic_relation as r2
              on r2.id = pr2.relation_id 

              where a.year_born in (select min (a3.year_born)

              from sociolinguistic_person as a3
              join sociolinguistic_personrelation as pr3
              on a3.id = pr3.person1_id 
              join sociolinguistic_person as b3
              on b3.id = pr3.person2_id 
              join sociolinguistic_relation as r3
              on r3.id = pr3.relation_id

              where r3.name != 'spouse'
              and b3.id = b.id)

              and r.name != 'spouse'
              and r2.name != 'spouse'

              order by 1, r1_name, r2_name
              ) q where sp.id = q.child_id
              or sp.id = q.parent_id
              or sp.id = q.grandparent_id
             ) q2

### Выбор родственных отношений между родителями и детьми

Создадим подзапрос, в котором исключим родственные отношения между супругами.

    WITH parent_relations AS(
    SELECT 
        a.person1_id AS child, 
        a.person2_id AS parent
    FROM sociolinguistic_personrelation AS a
        LEFT JOIN sociolinguistic_relation AS b
        ON a.relation_id=b.id
    WHERE b.name IN ('mother', 'father')
    )


### Обогащение информацией

* о годе рождения,
* о месте рождения,
* о месте и времени проведения опроса,
* о статусе is_present в каком-либо из мест.

    WITH full_data AS (
        SELECT 
            a.child,
            b.year_born AS child_year_born,
            d.place_born_id AS child_place_born_id,
            f.year AS child_year_questionnaire,
            f.place_id AS child_place_questionnaire,
            h.place_id AS child_present_place,
            a.parent,
            c.year_born AS parent_year_born,
            e.place_born_id AS parent_place_born_id,
            g.year AS parent_year_questionnaire,
            g.place_id AS parent_place_questionnaire,
            i.place_id AS parent_present_place
        FROM parent_relations AS a
            LEFT JOIN analytic_person_places AS b
            ON a.child=b.id
            LEFT JOIN analytic_person_places AS c
            ON a.parent=c.id
            LEFT JOIN sociolinguistic_person AS d
            ON a.child=d.id
            LEFT JOIN sociolinguistic_person AS e
            ON a.parent=e.id
            LEFT JOIN sociolinguistic_questionnaire AS f
            ON a.child=f.person_id
            LEFT JOIN sociolinguistic_questionnaire AS g
            ON a.parent=g.person_id
            LEFT JOIN sociolinguistic_personplaces AS h
            ON h.is_present=True AND a.child=h.person_id
            LEFT JOIN sociolinguistic_personplaces AS i
            ON i.is_present=True AND a.parent=i.person_id
    )

### Определение периодов для переезда и обогащение координатами

1. Различаются place_born_id и COALESCE(present_place, place_questionnaire):

        SELECT 
            child AS person, 
            child_place_born_id AS from_place,
            child_year_born AS from_year,
            COALESCE(child_present_place, child_place_questionnaire) AS to_place,
            child_year_questionnaire AS to_year
        FROM full_data
        WHERE 
            child_place_born_id IS NOT NULL
            AND COALESCE(child_present_place, child_place_questionnaire) IS NOT NULL
            AND child_year_born IS NOT NULL
            AND child_year_questionnaire IS NOT NULL
            AND child_place_born_id != COALESCE(child_present_place, child_place_questionnaire)

        UNION ALL

        SELECT 
            parent AS person, 
            parent_place_born_id AS from_place,
            parent_year_born AS from_year,
            COALESCE(parent_present_place, parent_place_questionnaire) AS to_place,
            parent_year_questionnaire AS to_year
        FROM full_data
        WHERE 
            parent_place_born_id IS NOT NULL
            AND COALESCE(parent_present_place, parent_place_questionnaire) IS NOT NULL
            AND parent_year_born IS NOT NULL
            AND parent_year_questionnaire IS NOT NULL
            AND parent_place_born_id != COALESCE(child_present_place, child_place_questionnaire)

2. Различаются parent_place_born_id и child_place_born_id
        
        UNION ALL

        SELECT
            parent AS person,
            parent_place_born_id AS from_place,
            parent_year_born AS from_year,
            child_place_born_id AS to_place,
            child_year_born AS to_year
        FROM full_data
        WHERE 
            parent_place_born_id IS NOT NULL
            AND parent_year_born IS NOT NULL
            AND child_place_born_id IS NOT NULL
            AND child_year_born IS NOT NULL
            AND parent_place_born_id != child_place_born_id 

3. Обогащение координатами и наименованием места, где united — все выбранные периоды.
    
        SELECT 
            a.person,
            a.from_place,
            b.longitude AS from_longitude,
            b.latitude AS from_latitude,
            b.name AS from_place_name,
            a.from_year,
            a.to_place,
            c.longitude AS to_longitude,
            c.latitude AS to_latitude,
            c.name AS to_place_name,
            a.to_year
            
        FROM united AS a
            LEFT JOIN sociolinguistic_place AS b
            ON a.from_place=b.id
            LEFT JOIN sociolinguistic_place AS c
            ON a.to_place=c.id

drop_duplicates

In [5]:
df = pd.read_sql("""
WITH full_data AS (

  SELECT 
      a.child,
      b.year_born AS child_year_born,
      d.place_born_id AS child_place_born_id,
      f.year AS child_year_questionnaire,
      f.place_id AS child_place_questionnaire,
      h.place_id AS child_present_place,
      a.parent,
      c.year_born AS parent_year_born,
      e.place_born_id AS parent_place_born_id,
      g.year AS parent_year_questionnaire,
      g.place_id AS parent_place_questionnaire,
      i.place_id AS parent_present_place
  FROM (SELECT 
            a.person1_id AS child, 
            a.person2_id AS parent
        FROM sociolinguistic_personrelation AS a
            LEFT JOIN sociolinguistic_relation AS b
            ON a.relation_id=b.id
        WHERE b.name IN ('mother', 'father')) AS a
      LEFT JOIN analytic_person_places AS b
      ON a.child=b.id
      LEFT JOIN analytic_person_places AS c
      ON a.parent=c.id
      LEFT JOIN sociolinguistic_person AS d
      ON a.child=d.id
      LEFT JOIN sociolinguistic_person AS e
      ON a.parent=e.id
      LEFT JOIN sociolinguistic_questionnaire AS f
      ON a.child=f.person_id
      LEFT JOIN sociolinguistic_questionnaire AS g
      ON a.parent=g.person_id
      LEFT JOIN sociolinguistic_personplaces AS h
      ON h.is_present=True AND a.child=h.person_id
      LEFT JOIN sociolinguistic_personplaces AS i
      ON i.is_present=True AND a.parent=i.person_id
)

SELECT 
    a.person,
    a.from_place,
    b.longitude AS from_longitude,
    b.latitude AS from_latitude,
    b.name AS from_place_name,
    a.from_year,
    a.to_place,
    c.longitude AS to_longitude,
    c.latitude AS to_latitude,
    c.name AS to_place_name,
    a.to_year
FROM (
    SELECT 
         child AS person, 
         child_place_born_id AS from_place,
         child_year_born AS from_year,
         COALESCE(child_present_place, child_place_questionnaire) AS to_place,
         child_year_questionnaire AS to_year
     FROM full_data
     WHERE 
         child_place_born_id IS NOT NULL
         AND COALESCE(child_present_place, child_place_questionnaire) IS NOT NULL
         AND child_year_born IS NOT NULL
         AND child_year_questionnaire IS NOT NULL
         AND child_place_born_id != COALESCE(child_present_place, child_place_questionnaire)

     UNION ALL

     SELECT 
         parent AS person, 
         parent_place_born_id AS from_place,
         parent_year_born AS from_year,
         COALESCE(parent_present_place, parent_place_questionnaire) AS to_place,
         parent_year_questionnaire AS to_year
     FROM full_data
     WHERE 
         parent_place_born_id IS NOT NULL
         AND COALESCE(parent_present_place, parent_place_questionnaire) IS NOT NULL
         AND parent_year_born IS NOT NULL
         AND parent_year_questionnaire IS NOT NULL
         AND parent_place_born_id != COALESCE(child_present_place, child_place_questionnaire)

    UNION ALL

    SELECT
         parent AS person,
         parent_place_born_id AS from_place,
         parent_year_born AS from_year,
         child_place_born_id AS to_place,
         child_year_born AS to_year
     FROM full_data
     WHERE 
         parent_place_born_id IS NOT NULL
         AND parent_year_born IS NOT NULL
         AND child_place_born_id IS NOT NULL
         AND child_year_born IS NOT NULL
         AND parent_place_born_id != child_place_born_id) AS a
        LEFT JOIN sociolinguistic_place AS b
        ON a.from_place=b.id
        LEFT JOIN sociolinguistic_place AS c
        ON a.to_place=c.id
""", conn)

In [6]:
df = df.drop_duplicates()

### Проверка данных
Проверим, есть ли повторяющиеся передвижения.

In [7]:
df.groupby(['person', 'from_place', 'to_place']).count()['from_year'].unique()

array([1], dtype=int64)

In [8]:
df.groupby(['person', 'from_place']).count()['from_year'].unique()

array([1], dtype=int64)

In [9]:
df.groupby(['person', 'to_place']).count()['from_year'].unique()

array([1], dtype=int64)

In [10]:
df.groupby(['person']).count()['from_year'].unique()

array([1], dtype=int64)

Нет, каждое перемещение отображает отдельного человека, наложения отсутствуют.

Проверим на наличие ошибок:
    * совпадают место отправления и место назначения
    * перепутаны местами from_year и to_year

In [11]:
df[df['from_place'] == df['to_place']]

,person,from_place,from_longitude,from_latitude,from_place_name,from_year,to_place,to_longitude,to_latitude,to_place_name,to_year


In [12]:
df[df['from_year'] > df['to_year']]

,person,from_place,from_longitude,from_latitude,from_place_name,from_year,to_place,to_longitude,to_latitude,to_place_name,to_year


Проверим полноту данных.

In [13]:
df[(df['from_longitude'].isna())
  |(df['from_latitude'].isna())
  |(df['to_longitude'].isna())
  |(df['to_latitude'].isna())]

,person,from_place,from_longitude,from_latitude,from_place_name,from_year,to_place,to_longitude,to_latitude,to_place_name,to_year
0,37088,4869,NaN,NaN,Устье,1960,4864,143.1078,49.2388,Поронайск,2017
2,37097,4872,NaN,NaN,Речное,1952,4864,143.1078,49.2388,Поронайск,2017
16,37232,4898,NaN,NaN,"Минабэцу (Птичье), Корсаковский р-н",1936,4864,143.1078,49.2388,Поронайск,2017
20,37284,4907,NaN,NaN,"с. Симаси, Комсомольский р-н, Хабаровский край",1932,4864,143.1078,49.2388,Поронайск,2017
24,37302,4911,NaN,NaN,"с. Падали (ныне город Амурск), Хабаровский край",1933,4864,143.1078,49.2388,Поронайск,2017
...,...,...,...,...,...,...,...,...,...,...,...
1579,41352,5459,NaN,NaN,Хайзаран,1875,5457,NaN,NaN,"пос. Хайзаран, Баяндаевский р-н",1905
1581,41363,5210,NaN,NaN,Алтайский край,1944,3179,105.8799,53.9612,Качуг,1974
1582,41366,4878,NaN,NaN,Сахалин,1914,5210,NaN,NaN,Алтайский край,1944
1583,41368,5462,NaN,NaN,Европейская часть России,1914,5461,71.5082,56.0052,"Крутинка, Омская область",1944


Данные неполные. Выгрузим id мест, о которых нет данных, и попробуем заполнить их координаты вручную.

In [14]:
from_places = df[(df['from_longitude'].isna())
  |(df['from_latitude'].isna())]['from_place'].unique().tolist()
to_places = df[(df['to_longitude'].isna())
  |(df['to_latitude'].isna())]['to_place'].unique().tolist()
places = from_places + to_places
places = set(places)

In [15]:
print('Количество мест с незаполненными координатами: {}]'.format(len(places)))

Количество мест с незаполненными координатами: 305]


In [16]:
all_places = df['from_place'].unique().tolist() + df['to_place'].unique().tolist()
all_places = set(all_places)
print('Всего уникальных мест: {}'.format(len(all_places)))

Всего уникальных мест: 522


Локально соберем общую сводку с местами.

In [17]:
missed_coordinates = pd.read_excel('missed_coordinates.xlsx')

In [18]:
missed_coordinates.head(5)

,id,name,latitude,longitude,is_coords_approximate,comment
0,5139,0,NaN,NaN,NaN,?
1,5363,?,NaN,NaN,NaN,?
2,5372,[посёлок] на р. Лена,NaN,NaN,NaN,не определить
3,5443,60 км. От Ербогачена,NaN,NaN,NaN,не определить
4,5107,Авдотья: Кислокан; Марина: Хамакар,NaN,NaN,NaN,ошибка данных


In [19]:
coordinates_from = df[['from_place', 'from_longitude', 'from_latitude', 'from_place_name']].rename(columns={
    'from_place': 'id', 
    'from_longitude': 'longitude', 
    'from_latitude': 'latitude',
    'from_place_name': 'name'
})
coordinates_to = df[['to_place', 'to_longitude', 'to_latitude', 'to_place_name']].rename(columns={
    'to_place': 'id', 
    'to_longitude': 'longitude', 
    'to_latitude': 'latitude',
    'to_place_name': 'name'
})
coordinates = pd.concat([coordinates_from, coordinates_to]).drop_duplicates()
coordinates = coordinates[coordinates['longitude'].isna() == False]
coordinates.shape

(217, 4)

In [20]:
coordinates = pd.concat([missed_coordinates, coordinates])

In [21]:
coordinates = coordinates[coordinates['longitude'].isna() == False].reset_index(drop=True)
coordinates.shape

(448, 6)

Всего уникальных координатных точек.

In [22]:
coordinates.groupby(['latitude', 'longitude']).count().shape

(362, 4)

In [21]:
coordinates.to_excel('coordinates.xlsx', index=False)

Просмотрим наименования мест в общей таблице и отредактируем вручную.

Соберем новую таблицу с периодами.

In [23]:
coordinates = pd.read_excel('coordinates_edited.xlsx')

In [24]:
coordinates_from = coordinates.rename(columns={
    'id':'from_place',
    'latitude': 'from_latitude',
    'longitude': 'from_longitude',
    'name': 'from_place_name'
})[['from_place', 'from_latitude', 'from_longitude', 'from_place_name']]
coordinates_to = coordinates.rename(columns={
    'id':'to_place',
    'latitude': 'to_latitude',
    'longitude': 'to_longitude',
    'name': 'to_place_name'
})[['to_place', 'to_latitude', 'to_longitude', 'to_place_name']]

df.drop(['from_latitude', 'from_longitude', 'from_place_name', 
         'to_latitude', 'to_longitude', 'to_place_name'], inplace=True, axis=1)
df = df.merge(coordinates_from, on='from_place', how='left')
df = df.merge(coordinates_to, on='to_place', how='left')

In [25]:
df = df[['person', 'from_place', 'from_latitude', 'from_longitude', 'from_place_name', 'from_year', 
        'to_place', 'to_latitude', 'to_longitude', 'to_place_name', 'to_year']]

In [26]:
df[(df['from_latitude'].isna() == False)
  &(df['to_longitude'].isna() == False)]

,person,from_place,from_latitude,from_longitude,from_place_name,from_year,to_place,to_latitude,to_longitude,to_place_name,to_year
1,37097,4872,49.470735,143.710445,Речное,1952,4864,49.238800,143.107800,Поронайск,2017
2,37106,4874,46.632500,142.797700,Корсаков,1942,4864,49.238800,143.107800,Поронайск,2017
3,37115,4877,49.305800,143.735800,пос. Нева,1955,4864,49.238800,143.107800,Поронайск,2017
4,37124,4880,52.340278,143.065000,пос. Вал,1957,4864,49.238800,143.107800,Поронайск,2017
5,37151,4884,48.201700,142.540346,Пугачево,1965,4864,49.238800,143.107800,Поронайск,2017
...,...,...,...,...,...,...,...,...,...,...,...
1336,41350,5458,52.315000,109.868200,Алан,1905,5454,53.578890,105.990560,"совхоз Фрунзе, улус Степно-Болтай, Качугский р-н",1935
1337,41352,5459,53.302086,106.001114,пос. Кайзаран,1875,5457,53.302086,106.001114,пос. Кайзаран,1905
1338,41364,5461,56.005200,71.508200,Крутинка,1944,3179,53.961200,105.879900,Качуг,1974
1339,41363,5210,52.230105,81.089545,Алтайский край,1944,3179,53.961200,105.879900,Качуг,1974


In [27]:
df[(df['from_latitude'].isna() == False)
  &(df['to_longitude'].isna() == False)]['person'].unique().shape

(1083,)

### Дополнительная информация
Хотим выгрузить информацию о годах рождения респондентов.

In [35]:
people = df[(df['from_latitude'].isna() == False)
  &(df['to_longitude'].isna() == False)]['person'].unique().tolist()
people = [str(i) for i in people]
len(people)

1083

In [41]:
people_df = pd.read_sql("""
SELECT id, year_born
FROM analytic_person_places
WHERE id IN ('{}')
""".format("', '".join(people)), conn)

In [42]:
people_df[people_df['year_born'].isna()]

,id,year_born


In [45]:
people_df.to_excel('people_year_born.xlsx', index=False)